In [ ]:
import numpy as np

# Function to read the input dataset as per the prescribed format in the feature map.
def readGermanCreditDataset(fname):
    inputData = []
# inputData = np.fromfile("german.adcg.trainingd", dtype=dt)#np.loadtxt("german.adcg.trainingd", comments="#", delimiter=",", unpack=False)#np.loadtxt('german.adcg.trainingd')
    f = open(fname,'r')
    featureMap = {'A11':1,'A12':2,'A13':3,'A14':4,
                  'A30':1,'A31':2,'A32':3,'A33':4,'A34':5,
                  'A40':1,'A41':2,'A42':3,'A43':4,'A44':5, 'A45':6,'A46':7,'A47':8,'A48':9,'A49':10, 'A410':0,
                  'A61':2,'A62':3,'A63':4,'A64':5,'A65':1,
                  'A71':1,'A72':2,'A73':3,'A74':4,'A75':5,
                  'A91':1,'A92':4,'A93':2,'A94':3,'A95':4,
                  'A101':1,'A102':2,'A103':3,
                  'A121':4,'A122':3,'A123':2,'A124':1,
                  'A141':1,'A142':2,'A143':3,
                  'A151':2,'A152':3,'A153':1,
                  'A171':1,'A172':2,'A173':3, 'A174':4,
                  'A191':1,'A192':2,
                  'A201':1,'A202':2}
    for line in f:
    #     inputLine = np.empty(shape=(1,20))
        inputLine = []
        numericalData = []
        for word in line.split():
            if word in featureMap:
    #             print featureMap[word]
                inputLine.append(featureMap[word])
            else:
#                 inputLine.append(int(word))
                numericalData.append(int(word))
#         inputData.append(inputLine)
        inputData.append((inputLine + numericalData))
    f.close()
#     print inputData[0]
    inputDatanp = np.array(inputData)

    return inputDatanp


#print trainedLabels[:3]


# Function to write predicted labels for the Test dataset to a file.
def writePrdictedLabelFile(YPred):
    f = open("german.adcg.testing.label","w")
    f.write("Id,Prediction" + "\n")

    for i in xrange(len(YPred)):
        f.write(str(i+1) + "," + str(YPred[i])+ "\n")
    
    f.close()

# Read the input data and labels
XTrain = readGermanCreditDataset("german.adcg.trainingd")
XTest = readGermanCreditDataset("german.adcg.testingd")
Y = np.loadtxt("german.adcg.training.label", delimiter=",", skiprows=1, usecols=(1,))

from sklearn.preprocessing import OneHotEncoder
encTrain1 = OneHotEncoder()
encTrain = encTrain1.fit_transform(XTrain[:,:12]).toarray()

encTest1 = OneHotEncoder()
encTest = encTest1.fit_transform(XTest[:,:12]).toarray()

encNTrain = XTrain[:,13:]
encNTest = XTest[:,13:]
print XTest.shape, XTrain.shape
print encNTest.shape, encNTrain.shape

# XTrain = np.concatenate((encTrain, encNTrain), axis=1)
# XTest = np.concatenate((encTest, encNTest), axis=1)

#Normalizing & scaling of data
from sklearn import preprocessing
# encNTrain = preprocessing.scale(encNTrain)
encNTrain = (encNTrain - encNTrain.mean())/encNTrain.std()
# encNTest = preprocessing.scale(encNTest)
encNTest = (encNTest - encNTest.mean())/encNTest.std()

#c = XTrain.reshape((500, 52))
XTrain = np.append(encTrain,encNTrain,axis=1)
XTest = np.append(encTest,encNTest,axis=1)

print XTrain.shape
print XTest.shape


#Split Training Data into Train & Validation
from sklearn.cross_validation import train_test_split
np.random.seed(9)
#X_train, X_test, y_train, y_test = train_test_split(XTrain[:,:12], Y, test_size=0.8)
X_train, X_test, y_train, y_test = train_test_split(XTrain, Y, test_size=0.8)
#X_train, X_test, y_train, y_test = train_test_split(encNTrain, Y, test_size=0.8)
XTrain = X_train
XTest = X_test
Y = y_train
YTest = y_test

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

train_error = np.empty(5)
test_error = np.empty(5)
for degree in xrange(1,6):
    est = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    est.fit(X_train, y_train)
    train_error[degree-1] = mean_squared_error(y_train, est.predict(X_train))
    test_error[degree-1] = mean_squared_error(y_test, est.predict(X_test))
print train_error
print test_error

plt.plot(np.arange(5), train_error, color='green', label='train')
plt.plot(np.arange(5), test_error, color='red', label='test')
plt.ylim((0.0, 1e0))
plt.ylabel('log(mean squared error)')
plt.xlabel('degree')
plt.legend(loc='lower left')
plt.show()

# Linear Regression

from sklearn import linear_model
# from sklearn.decomposition import PCA

# pca = PCA(n_components=48)
# print XTrain.shape
# pca.fit(XTrain)
# XTrain = pca.transform(XTrain)
# pca.fit(XTest)
# XTest = pca.transform(XTest)
# print XTrain.shape
LR = linear_model.LinearRegression()
LR.fit(XTrain, Y)
# # LR.fit(encTrain, Y)

# Finds the optimal model parameters using a least squares method.

# To get the parameter values:
LR.get_params()

# To predict a new input XTest,
YPred = LR.predict(XTest)
# YPred = LR.predict(encTest)


writePrdictedLabelFile(YPred)

# # Regularization - Tune the parameter lambda to prevent overfitting.


# # R^2 statistics to observe the goodness of a line fitting.
# LR.score(XTest, Y)

# # Submission Score 0.61265



# # Ridge
# clfR = linear_model.RidgeCV(alphas=[0.1, 0.3, 0.7, 1.0, 1.3, 1.7, 2, 2.3, 5, 7, 15, 50, 100])
# clfR.fit(XTrain, Y)
# print clfR.alpha_


# # Lasso
# clflml = linear_model.Lasso(alpha = 0.3)

# Bayesian
# clflml = linear_model.BayesianRidge()
# clflml.fit(XTrain, Y)
# YPred = clflml.predict(XTest)
# writePrdictedLabelFile(YPred)

# ARDRegression
clflml = linear_model.ARDRegression(compute_score=True)
clflml.fit(XTrain, Y)
YPred = clflml.predict(XTest)
writePrdictedLabelFile(YPred)



# # Logistic Regression

# LogReg = linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None)

# LogReg.fit(XTrain, Y)

# # Finds the optimal model parameters using a least squares method.

# # To get the parameter values:
# LogReg.get_params()

# # To predict a new input XTest,
# YPred = LogReg.predict(XTest)

# writePrdictedLabelFile(YPred)

# # Submission Score 0.54007




# # Support Vector Machines (SVM)


# from sklearn import svm

# clf = svm.SVC()
# clf.fit(XTrain, Y)  

# # To predict a new input XTest
# YPred = clf.predict(XTest)

# writePrdictedLabelFile(YPred)

# # Submission Score 0.50000




# # Multi-class classification - Linear SVM

# lin_clf = svm.LinearSVC()
# lin_clf.fit(XTrain, Y)

# # To predict a new input XTest
# YPred = lin_clf.predict(XTest)

# writePrdictedLabelFile(YPred)

# # Submission Score 0.58125



# # Non Linear SVM

# clfnl = svm.NuSVC()
# clfnl.fit(XTrain, Y)

# # To predict a new input XTest
# YPred = clfnl.predict(XTest)

# writePrdictedLabelFile(YPred)

# # Submission Score 0.50037



# SVM Kernels
# print 'SVM Starting'
# pca = PCA(n_components=48)
# pca.fit(XTrain)
# XTrain = pca.transform(XTrain)
# pca.fit(XTest)
# XTest = pca.transform(XTest)
# # print XTrain.shape
# linear_svc = svm.SVC(kernel='linear')
# print linear_svc.kernel
# linear_svc.fit(XTrain, Y)
# print 'SVM Starting'
# # To predict a new input XTest
# YPred = linear_svc.predict(XTest)

# writePrdictedLabelFile(YPred)


# rbf_svc = svm.SVC(kernel='rbf')
# print rbf_svc.kernel
# rbf_svc.fit(XTrain, Y)
# # print 'SVM Starting'
# # To predict a new input XTest
# YPred = rbf_svc.predict(XTest)

# writePrdictedLabelFile(YPred)




# # Ridge Regression
# from sklearn import linear_model

# # Setting the regularization parameter: generalized Cross-Validation

# from sklearn import linear_model
# clfRidge = linear_model.RidgeCV(alphas=[0.1, 1.0, 10.0])
# clfRidge.fit(XTrain, Y)       

# alpha = clfRidge.alpha_ 
# print alpha 

# clfRidge = linear_model.Ridge(alpha)
# clfRidge.fit(XTrain, Y) 

# print clfRidge.coef_
# print clfRidge.intercept_ 


##Decision Tree
# from sklearn import tree
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(XTrain, Y)
# YPred = clf.predict(XTest)
# writePrdictedLabelFile(YPred)


##Decision Tree with Dimensionality Reduction
from sklearn.decomposition import PCA
# from sklearn import tree
# # pca = PCA(n_components=48)
# # pca.fit(XTest)
# # XTest = pca.transform(XTest)
# # pca.fit(XTrain)
# # XTrain = pca.transform(XTrain)
# clf = tree.DecisionTreeClassifier()
# clf = clf.fit(XTrain, Y)
# YPred = clf.predict(XTest)
# writePrdictedLabelFile(YPred)


##Random Forests
# from sklearn.ensemble import RandomForestClassifier
# pca = PCA(n_components=48)
# pca.fit(XTest)
# XTest = pca.transform(XTest)
# pca.fit(XTrain)
# XTrain = pca.transform(XTrain)
# clf = RandomForestClassifier(n_estimators=10)
# clf = clf.fit(XTrain, Y)
# YPred = clf.predict(XTest)
# writePrdictedLabelFile(YPred)

print 'completed'




# import matplotlib.pyplot as plt
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import Ridge
    
# fig, ax_rows = plt.subplots(4, 2, figsize=(8, 10))

# def plot_coefficients(est, ax, label=None, yscale='log'):
#     coef = est.steps[-1][1].coef_.ravel()
#     if yscale == 'log':
#         ax.semilogy(np.abs(coef), marker='o', label=label)
#         ax.set_ylim((1e-1, 1e8))
#     else:
#         ax.plot(np.abs(coef), marker='o', label=label)
#     ax.set_ylabel('abs(coefficient)')
#     ax.set_xlabel('coefficients')
#     ax.set_xlim((1, 9))

# degree = 9
# alphas = [0.0, 1e-8, 1e-5, 1e-1]
# for alpha, ax_row in zip(alphas, ax_rows):
#     ax_left, ax_right = ax_row
#     est = make_pipeline(PolynomialFeatures(degree), Ridge(alpha=alpha))
#     est.fit(X_train, y_train)
#     plot_approximation(est, ax_left, label='alpha=%r' % alpha)
#     plot_coefficients(est, ax_right, label='Ridge(alpha=%r) coefficients' % alpha)

# plt.tight_layout()




In [6]:
writePrdictedLabelFile(YPred)